PHASE TWO: Adding Latitude and Longitude Coordinates

    Objective: Find the geographic location of each headline in latitude and longitude coordinates from the city/country names.

Workflow:

Step One

    import necessary modules: geonamescache/numpy/pandas/unidecode

In [6]:
import geonamescache
import numpy as np
import pandas as pd
from unidecode import unidecode

Step Two

    Load in the headline data and examine it for any data quality issues.

In [7]:
# load the extracted data of which is the output of phase1
df = pd.read_csv('database/extracted_data.csv')

# bring in external data through geonamescache and initialize necessary lists

gnc = geonamescache.GeonamesCache()
countries_info = gnc.get_countries()
cities_info = gnc.get_cities()
list_longitude = []
list_latitude = []

# remove the empty lines in the dataframe
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)

Step Three

    Using regular expressions and the cities and countries within the geonamescache library, match any cities/countries within each headline.

In [8]:
# get country code according to its name
def get_countrycode_from_name(country_name):
    for code, country_entry_value in countries_info.items():
        if unidecode(country_entry_value['name']).strip() == country_name:
            return code
    return None


df['countrycode'] = df['country'].apply(get_countrycode_from_name)

In [9]:
# match the longitude and latitude of the city
city_column = df['city']
countrycode_column = df['countrycode']
for index in df.index:
    print(f'\r{index}', end='')
    city_matches = {}
    for city_ID, city_entry_value in cities_info.items():
        if city_entry_value['countrycode'] == countrycode_column[index]:
            # consider to match the official name first, then consider to match the alternate names
            if unidecode(city_entry_value['name']).strip() == city_column[index]:
                city_matches[city_ID] = [city_entry_value['longitude'], city_entry_value['latitude'],
                                         len(city_entry_value['alternatenames']), city_entry_value['population']]
            else:
                for alternate in city_entry_value['alternatenames']:
                    if unidecode(alternate).strip() == city_column[index]:
                        city_matches[city_ID] = [city_entry_value['longitude'], city_entry_value['latitude'],
                                                 len(city_entry_value['alternatenames']),
                                                 city_entry_value['population']]
    # For cities with same city name (whether it's official name or alternate name), we take the one which has most alternate names. If there are still multiple matches, we choose the one which has most population.
    # We assume that the more alternate names and population the city has, the greater influence it has.
    max_alternate = 0
    for ID, info in city_matches.items():
        max_alternate = max(max_alternate, info[2])
    max_population = 0
    right_longitude = np.nan
    right_latitude = np.nan
    for ID, info in city_matches.items():
        if info[2] == max_alternate:
            if info[3] > max_population:
                max_population = info[3]
                right_longitude = info[0]
                right_latitude = info[1]
    list_longitude.append(right_longitude)
    list_latitude.append(right_latitude)

622

Step Four

    Put the extracted data into a pandas DataFrame with three columns: headline, city, country

    

In [10]:
# add longitude and latitude to the original DataFrame
df['longitude'] = list_longitude
df['latitude'] = list_latitude

# post processing
df = df[df.columns[[0, 1, 2, 4, 5, 3]]]  # readjust the sequence of columns
print(df[df.isnull().T.any()])  # print the rows with null data
df.dropna(inplace=True)  # drop rows which contains null data
df.to_csv('database/match_data.csv', index=False)
print(df.info())
print(df.describe())

                         headline       city        country  longitude  \
17  Louisiana Zika cases up to 26  Louisiana  United States        NaN   

    latitude countrycode  
17       NaN          US  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 622 entries, 0 to 622
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   headline     622 non-null    object 
 1   city         622 non-null    object 
 2   country      622 non-null    object 
 3   longitude    622 non-null    float64
 4   latitude     622 non-null    float64
 5   countrycode  622 non-null    object 
dtypes: float64(2), object(4)
memory usage: 34.0+ KB
None
        longitude    latitude
count  622.000000  622.000000
mean   -37.647664   26.651596
std     80.026700   20.701847
min   -156.506040  -53.787690
25%    -91.444495   14.753155
50%    -76.315220   33.662735
75%      7.702465   40.658300
max    179.364510   59.938630
